## Abstract  

我们提出了一个概念上简单，灵活，通用的对象实例分割框架。我们的方法有效地检测图像中的对象，同时为每个实例生成高质量的分割掩膜。该方法被称为 Mask R-CNN，通过增加一个用于预测对象掩码的分支，并与现有的用于边界框识别的分支相结合，来扩展 faster R-CNN。Mask R-CNN 训练简单，以5 fps的速度运行,相比 faster R-CNN 只增加了一小部分的运算时间。此外，Mask R-CNN 很容易推广到其他任务，例如允许我们在相同的框架下估计人的姿势。我们在 COCO 的三个数据集上都表现出了 state of art 的效果，包括实例分割，边界框对象检测和人体关键点检测。在没有任何技巧的情况下， Mask R-CNN 的效果都由于目前所有在单个任务上的方法，包括 COCO 2016 挑战的获胜者。希望我们简单而有效的方法能够作为一个 base line，并有助于简化未来在实例识别方面的研究。

## 1. Introduction  

视觉社区在短时间内迅速提高了目标检测和语义分割的效果。在很大程度上，这些进步是由强大的 base line 系统驱动的，例如分别用于目标检测和语义分割的 faster RCNN 和全卷积网络（FCN）框架。这些方法在概念上是直观的，并提供了灵活性和健壮性，以及快速的训练和预测时间。我们在这项工作中的目标是开发一个可比较的实例分割框架。  

实例分割是具有挑战性的，因为它需要正确检测图像中的所有对象，同时精确地分割每个实例。 因此，它将经典的计算机视觉任务中的目标检测和语义分割结合起来，目标检测的目标是对各个对象进行分类，并使用边界框对定位出每个目标的位置，语义分割的目标是在不区分每个实例的情况下将每个像素分类为固定的一组类别。鉴于此，可能需要一个复杂的方法才能获得好的结果。 然而，我们表明，一个令人惊讶的简单、灵活、快速的系统可以超越先前的实例分割结果。  

我们的方法称为 Mask R-CNN，通过在现有的分类和边界框回归上并行的在每个感兴趣区域（RoI）上添加一个用于预测分割掩码的分支来扩展 Faster R-CNN。掩模分支是应用于每个 ROI 的小 FCN，以 pixel-to-pixel 的方式预测分割掩模。基于 Faster R-CNN 框架，可以很方便的 构建 mask R-CNN 框架。另外，掩码分支只增加了一个小的计算开销，实现了一个快速的系统并可以进行快速的实验。  

Mask R-CNN 是 Faster R-CNN 的一个直观的扩展，但是正确地构建掩模分支对于获得好的结果是至关重要的。最重要的是，Faster R-CNN 并不是针对网络输入和输出之间的像素对像素的转换而设计的。最明显的是 RoIPool 为特征提取执行粗略的空间量化。为了解决这个错位问题，我们提出了一个简单的，无量化的 RoIAlign 层，保留了精确的空间位置信息。尽管是一个看似微小的变化，但 RoIlign 具有很大的影响力：它将掩模精度提高了10％到50％，在更严格的定位指标下显示出更大的收益。其次，我们发现将掩码和类别预测分离是非常重要的：我们独立预测每个类别的二进制掩码，而不需要在类别间进行竞争，并且依靠网络的 ROI 分类分支来预测类别。相比之下，FCNs 通常结合分割和分类执行每像素多类别分类，并且在我们的实验中这种方法在实例分割方面表现不佳。  

Mask R-CNN 在 COCO 实例分割任务中超越了之前所有先进的 single-model 结果。作为副产品，我们的方法也擅长 COCO 物体检测任务。在对照实验中，我们评估了多个基本实例，这使我们能够展示它的稳健性并分析核心因素的影响。  

我们的模型可以在 GPU 上以每帧大约 200ms 的速度运行，并且在单个 8-GPU 计算机上对 COCO 进行一到两天的训练。我们相信，快速训练和测试速度，以及灵活性和准确性的框架，将有益于并且简化未来对实例分割的研究。  

最后，我们通过 COCO 关键点数据集上的人体姿态估计任务展示了我们框架的通用性。通过将每个关键点视为一个单独的二进制掩码，只需最小的修改即可应用 Mask R-CNN 来检测特定实例的姿势。在没用任何技巧的情况下，Mask R-CNN超越了 2016 年 COCO 竞赛的胜利者，同时以5 fps的速度运行。因此Mask R-CNN 可以被更广泛地看作是实例级识别的灵活框架，并且可以容易地扩展到更复杂的任务。  

## 2. Related Work  

R-CNN：基于区域的 CNN（R-CNN）方法对边界框目标进行检测是为了管理候选目标的数量，并在每个ROI上独立评估卷积网络。R-CNN 允许在使用 RoIPool 的特征地图上参与 RoI，从而加快速度和精度。Fast R-CNN 通过学习区域建议网络（RPN）的注意机制来推进这一流程。Faster R-CNN 对于许多后续改进具有灵活性和可靠性，并且是当前在几个基准测试中主要的框架。  

Instance Segmentation：在RCNN有效性的驱动下，许多实例分割的方法都基于 segment proposals。早期的方法采用了自下而上的方法。DeepMask 和后续的作品学习提出候选分割区域，然后由 Fast R-CNN 进行分类。在这些方法中，分割先于识别，这是比较慢并且不太准确。 同样，戴等人提出了一个复杂的多阶段级联，先从 bounding-box proposals 中预测 segment proposals，然后进行分类。相反，我们的方法是基于掩码和类标签并行预测的，更简单灵活。  

最近，李等将 segment proposal system 和 object detection system 合并为“全卷积实例分割”（FCIS）。思想是全卷积预测一组位置敏感的输出通道。 这些通道同时进行分类、检测和分割，使系统更快。但 FCIS 在重叠实例上表现出系统性的错误并产生虚假边缘，表明它受到了分割实例的根本困难的挑战。

## 3. Mask R-CNN  

Mask R-CNN 在概念上是简单的：Faster R-CNN 对于每个候选对象具有两个输出，一个类别标签和一个边界框偏移;为此，我们添加一个输出对象掩码的第三个分支。因此 Mask R-CNN 是一个自然而直观的想法。但是 Mask 输出与分类和检测输出不同，需要提取更精细的目标空间布局。接下来，我们介绍 Mask R-CNN 的关键点，包括像素到像素的对齐，这是 Fast / Raster R-CNN 中没有的部分。  

Faster R-CNN：我们首先简要回顾一下 Faster R-CNN。 Faster R-CNN 由两个阶段组成。第一阶段称为区域提议网络（RPN），提出候选目标边界框。第二阶段，本质上是 Fast R-CNN，使用 RoIPool 从每个候选框中提取特征，并执行分类和边界框回归。这两部分可以共享权重，从而可以加快前向运行速度。  

Mask R-CNN：Mask R-CNN 采用相同的两阶段流程，第一阶段相同（即RPN）。在第二阶段，与预测类和边框偏移并行，Mask R-CNN 为每个 RoI 输出一个二进制掩码。这与最近的在 mask 预测之后进行分类的的系统是不同的。我们的方法遵循 Faster R-CNN 的思想，并行地应用了边界框分类和回归。  

在训练期间，我们在每个抽样的 ROI 上定义一个多任务损失函数 $L = L_{cls} + L_{box} + L_{mask}$。分类损失 $L_{cls}$ 和边界框损失 $L_{box}$ 与 Fast R-CNN 中定义相同。掩码分支对于每个 ROI 具有 $Km^2$ 维输出，编码 K 个分辨率为 m×m 的二进制掩码，每个代表一类。为此，对每个像素应用 sigmoid，并将 $L_{mask}$ 定义为平均二值交叉熵损失。ROI 仅在对应的 ground-truth 类别 k 上计算$L_{mask}$。  

我们对 $L_{mask}$ 的定义允许网络为类都生成一个 mask，而不需要类之间进行竞争。我们依靠专用分类分支来预测用于选择输出 mask 的类别标签。这个将标签与类别区分开来。这与将 FCN 应用于语义分割时的做法不同，语义分割通常使用每像素 softmax 和多项式交叉熵损失。在这种情况下，跨类别的 mask 进行竞争; 在我们的实验中，每像素 sigmoid 和二元损失，类间不存在竞争。通过实验显示，这一步在实例分割中很重要。  

Mask Representation：一个 mask 编码了一个输入目标的空间信息。因此，与通过全连接（fc）层不可避免地折叠成短输出矢量的类标签或框偏移不同，提取掩码的空间结构可以通过卷积提供的像素到像素的对应来自然地解决。  

具体而言，我们使用 FCN 预测每个 RoI 的 m×m 掩码。这允许掩码分支中的每个层保持显式的 m×m 对象空间布局而不将其折叠成缺少空间维度的向量表示。不同于之前fc 层用于掩模预测的方法，我们的全卷积表示需要更少的参数，并且如实验所证明的那样更准确。  

这种像素到像素的转换要求我们的 ROI 特征（其本身是小特征图）能够很好地对齐以忠实地保留显式的每像素空间对应。这促使我们开发了以下 RoAlign层，在掩码预测中扮演着重要的角色。  

RoIlign：RoIPool 是从每个 RoI 提取小特征图（例如7×7）的标准操作。RoIPool 先将浮点数 ROI 量化为离散的特征映射图，然后将这个量化的 RoI 细分为 spatial bins，最后聚合每个 bin 所涵盖的特征值（通常通过最大池）。这些量化引入了 RoI 和提取的特征之间的错位。虽然这可能不会影响分类，这对于小的转换是鲁棒的，但是对于预测像素的掩码有很大的负面影响。  

为了解决这个问题，我们提出一个 RoIlign 层，去除 RoIPool 的严格量化，正确对齐提取的特征和输入。我们提出的改变很简单：我们避免任何 RoI 边界或分组的量化（即我们使用 x/16 而不是[x / 16]）。我们使用双线性插值来计算每个 RoI bin 中四个规则采样坐标对应特征值的精确值，并将结果汇总（使用最大值或平均值）。    

Network Architecture：为了证明我们方法的通用性，我们实例化具有多种体系结构的 Mask R-CNN。为了清晰起见，我们区分：(i)用于整个图像上的特征提取的卷积体系结构，以及 (ii) 分别应用于每个 RoI 的边界框识别（分类和回归）和掩模预测的网络。  

我们用命名网络深度特征来表示主要网络架构。我们评估深度为 50 或 101 层的 ResNet 和 ResNeXt 网络。基于 ResNets 的 Faster R-CNN 第四阶段的卷积层提取特征，我们称之为 C4。 例如，ResNet-50的骨干网络由 ResNet-50-C4 表示。  

我们还探索了 Lin 等人最近提出的另一个更有效的结构。称为特征金字塔网络（FPN）。FPN使用具有横向连接的自顶向下的体系结构来从单一比例输入构建网络内的特征金字塔。Faster R-CNN 和 FPN 网络结构根据其规模从不同层次的特征金字塔中提取 RoI 特征。    

我们注意到我们的 Mask 分支有是一个简单的结构。更复杂的设计有改善性能的潜力，但不是这项工作的重点。  

### 3.1. Implementation Details  

我们用在 Fast/Faster R-CNN 工作中现有的超参数。虽然这些参数在原始文件是进行目标检测的，我们发现这些参数在实例分割系统上也是鲁棒的。  

Training：和 Fast R-CNN 一样，如果 ROI 与 ground-truth box 的 IOU 大于 0.5，就认为是正样本，否则是负样本。$L_{mask}$ 仅在正 RoI 上定义。掩码目标是 RoI 与其相关的 ground-truth mask 之间的交集。  

我们采用图像中心训练。调整图像的大小以使其比例（较短边缘）为800像素。在每个 GPU 上每个 mini-batch 有两个图像，每个图像有 N 个采样的RoI，比例为 1：3 的正负样本。C4 骨架的 N 为 64，FPN 为 512。我们在 8 个 GPU（有效小批量大小为16）上进行 160k 次迭代训练，学习率为 0.02，在 120k 迭代时减少 10。 我们使用 0.0001 的权重衰减和 0.9 的动量。    

Inference：测试时，C4 结构提取 300 个目标，FPN的提取 1000 个目标。我们对这些 proposals 进行边框回归，然后进行非极大值抑制。然后将掩码分支应用到最高得分的 100 个检测框上。虽然这不同于训练中使用的并行计算，但是它加快了 Inference 的速度，并提高了准确性（由于使用更少，更准确的RoI）。掩码分支可以预测每个 RoI 的 K 个掩码，但是我们只使用第 k 个掩码，其中 k 是分类分支预测的类别。然后将 m×m 浮点数掩码输出调整为 RoI 大小，并在阈值 0.5 下进行二进制化。  

请注意，由于我们只计算前100个检测框中的掩码，因此 Mask R-CNN 相比于 Faster R-CNN 只多出了很少的运行时间。  